In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [19]:
import xarray as xr
import torch
import math
import gpytorch
from matplotlib import pyplot as plt
from torch.utils.data import Dataset, DataLoader
import xbatcher
import tqdm
import pandas as pd

In [2]:
baker_url = 's3://petrichor/geosmart/baker.zarr/'
scg_url = 's3://petrichor/geosmart/scg.zarr/'

In [3]:
baker_ds = xr.open_dataset(baker_url, chunks='auto', engine='zarr', storage_options={"anon": True})
scg_ds = xr.open_dataset(scg_url, chunks='auto', engine='zarr', storage_options={"anon": True})

In [ ]:
# Convert time to numpy array
#years_tens = torch.tensor(years.values)

# Convert coordinates to tensors
#x_tens = torch.tensor(baker_array.coords['x'].values)
#y_tens = torch.tensor(baker_array.coords['y'].values)

In [4]:
bgen = iter(xbatcher.BatchGenerator(baker_ds, {"x": 50, "y": 50, "time": 55}))
chunk = next(bgen)

In [24]:
full_data_og = chunk.band1.to_dataframe().dropna().reset_index()

full_data_og.shape

(81921, 4)

In [27]:
full_data_og.to_csv("test_data_og.csv")

In [28]:
full_data = full_data_og

# Convert timestamps to unix time, normalize by starting at 0 and also do it in years
full_data['time'] = full_data['time'].astype('int64') / 1e9 / 60 / 60 / 24 / 365

# Start at 0
full_data['time'] = full_data['time'] - full_data['time'][0]

full_data

,time,y,x,band1
0,0.00000,5.409449e+06,580947.438712,706.601624
1,0.00000,5.409449e+06,580948.438712,707.278198
2,0.00000,5.409449e+06,580949.438712,707.780396
3,0.00000,5.409449e+06,580950.438712,708.258789
4,0.00000,5.409449e+06,580951.438712,708.737122
...,...,...,...,...
81916,73.29589,5.409400e+06,580992.438712,719.607239
81917,73.29589,5.409400e+06,580993.438712,720.352295
81918,73.29589,5.409400e+06,580994.438712,721.066284
81919,73.29589,5.409400e+06,580995.438712,721.751221


In [29]:
# Normalize x and y using StandardScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

full_data['x'] = scaler.fit_transform(full_data['x'].values.reshape(-1, 1))
full_data['y'] = scaler.fit_transform(full_data['y'].values.reshape(-1, 1))

In [79]:
# Partition full data into train and test sets. Do k fold cross validation on train set
# Use the first 80% of the data as the train set
def loadData(full_data, train_frac=0.8):
    train_data = full_data.iloc[:int(train_frac * full_data.shape[0])]
    test_data = full_data.iloc[int(train_frac * full_data.shape[0]):]

    # Train time
    train_time = torch.tensor(train_data['time'].values)
    train_x = torch.tensor(train_data['x'].values)
    train_y = torch.tensor(train_data['y'].values)
    train_elev = torch.tensor(train_data['band1'].values)

    # Test time
    test_time = torch.tensor(test_data['time'].values)
    test_x = torch.tensor(test_data['x'].values)
    test_y = torch.tensor(test_data['y'].values)
    test_elev = torch.tensor(test_data['band1'].values)

    # Stack train_time, train_x, train_y
    stacked_train = torch.stack([train_time, train_x, train_y], dim=1)

    # Stack test_time, test_x, test_y
    stacked_test = torch.stack([test_time, test_x, test_y], dim=1)

    # Cast as double
    stacked_train = stacked_train.double()
    train_elev = train_elev.double()
    stacked_test = stacked_test.double()
    test_elev = test_elev.double()

    return stacked_train, train_elev, stacked_test, test_elev

stacked_train, train_elev, stacked_test, test_elev = loadData(full_data)

# Experiments

### Simulation Data


### Exact GP
Let's first try a product RBF kernel in the form $\mathbf{K} = K_t \times K_x \times K_y$, where $K_t$ is the kernel for the time dimension, and $K_x$ and $K_y$ are the kernels for the spatial dimensions. 

In [65]:
stacked_train, train_elev, stacked_test, test_elev = loadData(full_data, 0.1)

In [66]:
class ExactSpatioTemporalGP(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(ExactSpatioTemporalGP, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=3))

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = ExactSpatioTemporalGP(stacked_train, train_elev, likelihood)

In [67]:
training_iter = 1

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(stacked_train)
    # Calc loss and backprop gradients
    loss = -mll(output, train_elev)
    loss.backward()
    optimizer.step()

### Sparse GP

In [63]:
stacked_train, train_elev, stacked_test, test_elev = loadData(full_data, 0.1)

In [64]:
from gpytorch.means import ConstantMean
from gpytorch.kernels import ScaleKernel, RBFKernel, InducingPointKernel
from gpytorch.distributions import MultivariateNormal

class SGPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(SGPRegressionModel, self).__init__(train_x, train_y, likelihood)
        self.mean_module = ConstantMean()
        self.base_covar_module = ScaleKernel(RBFKernel(ard_num_dims=3))
        self.covar_module = InducingPointKernel(self.base_covar_module, inducing_points=train_x[:1000, :].clone(), likelihood=likelihood)

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return MultivariateNormal(mean_x, covar_x)
    
# initialize likelihood and model
likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = SGPRegressionModel(stacked_train, train_elev, likelihood)

training_iter = 1

# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

for i in range(training_iter):
    # Zero gradients from previous iteration
    optimizer.zero_grad()
    # Output from model
    output = model(stacked_train)
    # Calc loss and backprop gradients
    loss = -mll(output, train_elev)
    loss.backward()
    optimizer.step()


/Users/ekinokos2/mambaforge/envs/gtpy/lib/python3.11/site-packages/linear_operator/utils/cholesky.py:40: NumericalWarning: A not p.d., added jitter of 1.0e-08 to the diagonal
  warnings.warn(


### LOVE

In [81]:
class LargeFeatureExtractor(torch.nn.Sequential):
    def __init__(self, input_dim):
        super(LargeFeatureExtractor, self).__init__()
        self.add_module('linear1', torch.nn.Linear(input_dim, 1000))
        self.add_module('relu1', torch.nn.ReLU())
        self.add_module('linear2', torch.nn.Linear(1000, 500))
        self.add_module('relu2', torch.nn.ReLU())
        self.add_module('linear3', torch.nn.Linear(500, 50))
        self.add_module('relu3', torch.nn.ReLU())
        self.add_module('linear4', torch.nn.Linear(50, 2))


class GPRegressionModel(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(GPRegressionModel, self).__init__(train_x, train_y, likelihood)

        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.GridInterpolationKernel(
            gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel(ard_num_dims=3)),
            grid_size=100, num_dims=3,
        )

        # Also add the deep net
        self.feature_extractor = LargeFeatureExtractor(input_dim=train_x.size(-1))

    def forward(self, x):
        # We're first putting our data through a deep net (feature extractor)
        # We're also scaling the features so that they're nice values
        projected_x = self.feature_extractor(x)
        projected_x = projected_x - projected_x.min(0)[0]
        projected_x = 2 * (projected_x / projected_x.max(0)[0]) - 1

        # The rest of this looks like what we've seen
        mean_x = self.mean_module(projected_x)
        covar_x = self.covar_module(projected_x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)


likelihood = gpytorch.likelihoods.GaussianLikelihood()
model = GPRegressionModel(stacked_train, train_elev, likelihood)

In [82]:
training_iterations = 10


# Find optimal model hyperparameters
model.train()
likelihood.train()

# Use the adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)  # Includes GaussianLikelihood parameters

# "Loss" for GPs - the marginal log likelihood
mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)


def train():
    #iterator = tqdm(range(training_iterations))
    for i in range(training_iterations):
        optimizer.zero_grad()
        output = model(stacked_train)
        loss = -mll(output, train_elev)
        loss.backward()
        #iterator.set_postfix(loss=loss.item())
        optimizer.step()

%time train()

RuntimeError: mat1 and mat2 must have the same dtype

### VNNGP

In [83]:
stacked_train, train_elev, stacked_test, test_elev = loadData(full_data, 0.1)

In [84]:
from gpytorch.models import ApproximateGP
from gpytorch.variational.nearest_neighbor_variational_strategy import NNVariationalStrategy


class GPModel(ApproximateGP):
    def __init__(self, inducing_points, likelihood, k=256, training_batch_size=256):

        m, d = inducing_points.shape
        self.m = m
        self.k = k

        variational_distribution = gpytorch.variational.MeanFieldVariationalDistribution(m)

        if torch.cuda.is_available():
            inducing_points = inducing_points.cuda()

        variational_strategy = NNVariationalStrategy(self, inducing_points, variational_distribution, k=k,
                                                     training_batch_size=training_batch_size)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ZeroMean()
        self.covar_module = gpytorch.kernels.MaternKernel(nu=2.5, ard_num_dims=d)

        self.likelihood = likelihood

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

    def __call__(self, x, prior=False, **kwargs):
        if x is not None:
            if x.dim() == 1:
                x = x.unsqueeze(-1)
        return self.variational_strategy(x=x, prior=False, **kwargs)

k = 256
training_batch_size = 64

likelihood = gpytorch.likelihoods.GaussianLikelihood()
# Note: one should use full training set as inducing points!
model = GPModel(inducing_points=stacked_train, likelihood=likelihood, k=k, training_batch_size=training_batch_size)

In [85]:
num_epochs = 10
num_batches = model.variational_strategy._total_training_batches

model.train()
likelihood.train()

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# Our loss object. We're using the VariationalELBO
mll = gpytorch.mlls.VariationalELBO(likelihood, model, num_data=train_elev.size(0))

for epoch in range(num_epochs):
    optimizer.zero_grad()
    output = model(x=None)
    # Obtain the indices for mini-batch data
    current_training_indices = model.variational_strategy.current_training_indices
    # Obtain the y_batch using indices. It is important to keep the same order of train_x and train_y
    y_batch = train_elev[...,current_training_indices]
    if torch.cuda.is_available():
        y_batch = y_batch.cuda()
    loss = -mll(output, y_batch)
    loss.backward()
    optimizer.step()

In [86]:
from torch.utils.data import TensorDataset, DataLoader

test_dataset = TensorDataset(stacked_test, test_elev)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

model.eval()
likelihood.eval()
means = torch.tensor([0.])
test_mse = 0
with torch.no_grad():
    for x_batch, y_batch in test_loader:
        preds = model(x_batch)
        means = torch.cat([means, preds.mean.cpu()])

        diff = torch.pow(preds.mean - y_batch, 2)
        diff = diff.sum(dim=-1) / stacked_test.size(0) # sum over bsz and scaling
        diff = diff.mean() # average over likelihood_nsamples
        test_mse += diff
means = means[1:]
test_rmse = test_mse.sqrt().item()

KeyboardInterrupt: 

In [78]:
test_rmse

tensor([692.5970, 693.1428, 693.7226,  ..., 721.0663, 721.7512, 722.3868])